In [1]:
import pandas as pd
import re
import json
from selenium.webdriver import Chrome
from collections import Counter
import urllib.request
from insta_scrape import get_posts, get_hashtags, get_image, get_full_info
import boto3
from io import BytesIO
from os import walk

### Hashtags to scrape: 
1. travel (Anna)
2. food (Anna)
3. animals (Anna)
4. selfie (Andy)
5. cars (Andy)
6. fitness (Andy)
7. babies (Alec)
8. wedding (Alec)
9. nature (Alec)
10. architecture

### Scraping:
Hashtags (from comments), Image Link, Image Name, Post Link, and Input Hashtag.  Then exporting the data to .json files, stored in a seperate folder.

*get_full_info* function also retrieves and downloads the actual image, which are stored locally to optimize possible space constraints.

In [ ]:
# hashtags = ['travel', 'food', 'animals']
# hashtags = ['selfie', 'cars', 'fitness']
hashtags = ['babies', 'wedding', 'nature', 'architecture']

In [ ]:
for hashtag in hashtags:
    hashtag_data = get_full_info(hashtag, 300)

    with open(f'json_files/{hashtag}.json', 'w') as f:
        json.dump(hashtag_data, f)

### Import JSON files back into pandas dataframe

In [ ]:
travel_df = pd.read_json('json_files/travel.json')
travel_df.head()

### Upload images from common folder to S3 bucket folders by hashtag (Anna)
Scraped images were all in one folder independently of the category. Had to separate them by hashtag

In [ ]:
#hashtags to separate
h_list = ['travel', 'food', 'animals']

#import json files
data = []
for h in h_list:    
    with open(f'json_files/{h}.json', 'r') as f:
        for line in f:
            data.append(json.loads(line))

In [ ]:
#combine json files for all 3 hashtags
data_full = data[0] + data[1] + data[2]
len(data_full)

In [ ]:
#access s3 with boto3 and upload images
s3 = boto3.resource('s3')

for h in h_list:
    for i in data_full:
        if i['search_hashtag'] == h:
            source = 'data/'+ i['image_local_name']
            bucket = 'instagram-images-mod4'
            destination = i['search_hashtag'] + '/' + i['image_local_name']
            s3.meta.client.upload_file(source, bucket, destination)

In [ ]:
f = []
for (dirpath, dirnames, filenames) in walk('data'):
    f.append(filenames)
    break

In [26]:
bucket="instagram-images-mod4"
key="travel/02c15b10-cf2c-4eaf-b3e5-cbbde14757c7.jpg"

#!pip install pillow
import numpy as np
from PIL import Image
from io import BytesIO

def fetch_image_from_s3(bucket, key):
    """Fetches an image from S3 and returns a numpy array"""
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket, Key=key)
    body = response['Body']
    data = body.read()    
    f = BytesIO(data)
    image = Image.open(f)   
    image_data = np.asarray(image)
    return image_data

In [29]:
fetch_image_from_s3(bucket, key).shape

(640, 640, 3)